# load data

In [ ]:
from scipy.io import loadmat

data = loadmat('all_monkey_data/one_ff_data/sessions_python.mat',
               squeeze_me=True,
               struct_as_record=False)

sessions = data['sessions_out']


## see one trial

In [ ]:
# indices
session_num = 0
trial_num = 0

# params
prs = default_prs()

# session / behaviour
session = data['sessions_out'][session_num]
behaviour = session.behaviour

# trials / stats
all_trials = behaviour.trials
all_stats = behaviour.stats
trial_ids = np.arange(len(all_trials))

trial = all_trials[trial_num]
stats = all_stats[trial_num]
pos_rel = stats.pos_rel

# continuous data
continuous = trial.continuous
print(continuous._fieldnames)

x = continuous.xmp
y = continuous.ymp
v = continuous.v
w = continuous.w
t = continuous.ts

# time step
prs.dt = round(np.mean(np.diff(t)), 5)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(x, y, 'k-')
plt.xlabel('x (forward)')
plt.ylabel('y (lateral)')
plt.axis('equal')


## neural data

In [ ]:
units = sessions[session_num].units
n_units = len(units)
 
# get unit0's trial_0 spike times
trial_neural_data = {}
for unit_id in range(len(sessions[session_num].units)):
    trial_neural_data[unit_id] = sessions[session_num].units[unit_id].trials[trial_num].tspk


# prepare data

In [ ]:
from neural_data_analysis.topic_based_neural_analysis.replicate_one_ff import one_ff_data_processing
from neural_data_analysis.topic_based_neural_analysis.replicate_one_ff import population_analysis_utils

covariate_names = [
    'v', 'w', 'd', 'phi',
    'r_targ', 'theta_targ',
    'eye_ver', 'eye_hor', 'move'
]

covariates_concat, trial_id_vec = population_analysis_utils.concatenate_covariates_with_trial_id(
    trials=all_trials,
    trial_indices=trial_ids,
    covariate_fn=lambda tr: one_ff_data_processing.compute_all_covariates(tr, prs.dt),
    time_window_fn=population_analysis_utils.full_time_window,
    covariate_names=covariate_names
)

covariates_concat['v'].shape

In [ ]:
Y = np.zeros((len(trial_id_vec), n_units))
for k in range(n_units):
    spk_counts, trial_id_vec_spk = population_analysis_utils.concatenate_trials_with_trial_id(
        all_trials,
        trial_ids,
        lambda tr, tid: population_analysis_utils.bin_spikes(
            trial_neural_data[k],
            tr.continuous.ts
        ),
        population_analysis_utils.full_time_window
    )
    Y[:, k] = spk_counts

Y_smooth = population_analysis_utils.smooth_signal(Y, prs.neural_filtwidth) / prs.dt

In [ ]:
all_events = {}
for event in ['t_targ', 't_move', 't_rew']:
    events_concat, trial_id_vec_evt = population_analysis_utils.concatenate_trials_with_trial_id(
        all_trials,
        trial_ids,
        lambda tr, tid: population_analysis_utils.event_impulse(tr, tid, event),
        population_analysis_utils.full_time_window
    )
    all_events[event] = events_concat
    
events_concat.shape

# PGAM

## temporal covariance

Temporal filters g were parameterized using a basis of ten raised cosine filters spanning a range of 600 milliseconds. The filter associated with target-onset was causal ([0, 600] ms), while the remaining filters were non-causal ([-300, 300] ms). Both spike-history filter h and coupling filter p were expressed using a basis of ten causal raised cosine filters in logarithmic time scale. Spike-history filters spanned 350 ms, while coupling filters spanned 1.375 seconds.

In [ ]:
unit_idx = 0
sm_handler = one_ff_pgam_design.build_smooth_handler_for_unit(
    unit_idx=unit_idx,                         # <-- choose the unit you want
    covariates_concat=covariates_concat,
    covariate_names=covariate_names,
    trial_id_vec=trial_id_vec,
    Y_binned=Y,
    all_events=all_events,
    dt=prs.dt,
    tuning_covariates=covariate_names,  # or a subset
    use_cyclic=set(),                  # e.g., {'heading_angle'}
    order=4,
)

sm_handler.smooths_var

# Call class

In [ ]:
binned_spikes_df = pd.DataFrame(Y, columns=np.arange(n_units))
runner = pgam_class.PGAMclass(x_var=binned_spikes_df, bin_width=prs.dt,
                                 save_dir='all_monkey_data/one_ff_data/pgam_results')

runner.sm_handler = sm_handler
runner.trial_ids = trial_id_vec
runner.train_trials = runner.trial_ids % 3 != 1
runner.run_pgam(neural_cluster_number=unit_idx)

In [ ]:
runner.kernel_h_length = 100
runner.post_processing_results()
runner.save_results()